In [1]:
import torch
import abmap

In [2]:
# Load AbMAP 
# Using Bepler-Berger as foundational model
abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_beplerberger_H.pt', plm_name='beplerberger')
abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_beplerberger_L.pt', plm_name='beplerberger')

# Using ESM1b
# abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_esm1b_H.pt', plm_name='esm1b')
# abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_esm1b_L.pt', plm_name='esm1b')

# Using ESM2
# abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_esm2_H.pt', plm_name='esm2')
# abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_esm2_L.pt', plm_name='esm2')

# Using ProtBert
# abmap_H = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_protbert_H.pt', plm_name='protbert')
# abmap_L = abmap.load_abmap(pretrained_path='../pretrained_models/AbMAP_protbert_L.pt', plm_name='protbert')

beplerberger loaded to cpu
Loaded the Pre-trained Model!
beplerberger loaded to cpu
Loaded the Pre-trained Model!


In [3]:
# Get embedding for one sequence
demo_seq = 'EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAGSYRSLFDYWGQGTLVTVSS'

# Contrastive augmentation (PLM, mutagenesis, CDR focus)
x = abmap.ProteinEmbedding(demo_seq, chain_type='H', embed_device=torch.device('cpu'))
x.create_cdr_specific_embedding(embed_type='beplerberger', k=50)

# # Pass the augmented embedding to AbMAP to get final embedding
with torch.no_grad():
    embed_var = abmap_H.embed(x.cdr_embedding.unsqueeze(0), embed_type='variable') # residue-level embeddings
    embed_fl = abmap_H.embed(x.cdr_embedding.unsqueeze(0), embed_type='fixed') # fixed-length

../data/anarci_files/temp0
Sequence: EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAGSYRSLFDYWGQGTLVTVSS
scheme: chothia


Error:  b'\nError: File existence/permissions problem in trying to open HMM file /Users/I0479386/opt/anaconda3/envs/abmap_main/lib/python3.9/site-packages/anarci/dat/HMMs/ALL.hmm.\nHMM file /Users/I0479386/opt/anaconda3/envs/abmap_main/lib/python3.9/site-packages/anarci/dat/HMMs/ALL.hmm not found (nor an .h\n\n'


IndexError: list index out of range

In [5]:
embed_var.shape, embed_fl.shape

NameError: name 'embed_var' is not defined

In [ ]:
# Get embeddings for a fasta file of sequences
# COMING VERY SOON

In [6]:
# Use ESM2 as foundational model
# COMING SOON

In [ ]:
# Fine-tuning AbMAP on your own functional data
# COMING SOON